In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [2]:
train_dir = "monkeyData/training/training"
test_dir = "monkeyData/validation/validation"
image_width = 224
image_height = 224
batch = 16

In [3]:
train_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                              rotation_range=30,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              fill_mode='nearest')
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

train_data = train_image_generator.flow_from_directory(train_dir, target_size=(image_height,image_width),batch_size=batch,)
test_data = test_image_generator.flow_from_directory(test_dir, target_size=(image_height,image_width),batch_size=batch)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [4]:
transfer_model = Sequential()
model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, 
                    input_shape=(image_height,image_width,3), pooling="avg")
#model.trainable = False
for layer in model.layers:
    layer.trainable = False

In [5]:
# model.summary()
# inputs = tf.keras.Input(shape=(image_height,image_width,3))
# x = model(inputs, training=False)
# x = layers.Dense(units=512,activation="relu")(x)
# output = layers.Dense(units=10, activation="softmax")(x)
# model = tf.keras.Model(inputs, output)
# model.summary()

model.summary()
transfer_model.add(model)
transfer_model.add(layers.Flatten())
transfer_model.add(layers.Dense(units=180, activation="relu"))
transfer_model.add(layers.Dense(units=10, activation="softmax"))
transfer_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [6]:
transfer_model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=['accuracy'])
train_accu_data = transfer_model.fit(train_data,epochs=5,steps_per_epoch=1098//batch)

Epoch 1/5
68/68 [==============================] - 36s 509ms/step - loss: 0.8048 - accuracy: 0.7532
Epoch 2/5
68/68 [==============================] - 34s 504ms/step - loss: 0.1613 - accuracy: 0.9538
Epoch 3/5
68/68 [==============================] - 35s 505ms/step - loss: 0.0762 - accuracy: 0.9787
Epoch 4/5
68/68 [==============================] - 36s 529ms/step - loss: 0.0597 - accuracy: 0.9871
Epoch 5/5
68/68 [==============================] - 36s 524ms/step - loss: 0.0348 - accuracy: 0.9935


In [ ]:
transfer_model.evaluate(test_data)

In [ ]:
plt.plot(train_accu_data.history['accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.grid()
plt.show()